In [1]:
import numpy as np
import xarray as xr
import glob as glob
import matplotlib.pyplot as plt
import cftime
import datetime as dt
import xesmf as xe

In [2]:
from dask.distributed import Client,LocalCluster
cluster = LocalCluster(n_workers=10, threads_per_worker=2) 
client = Client(cluster, asynchronous=True)  # start distributed scheduler locally.  Launch dashboard
client

/home/fiaz/anaconda3/envs/analysis_env/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43751 instead
  warnings.warn(
2023-06-13 10:30:33,188 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-ytxotvu8', purging
2023-06-13 10:30:33,189 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-6xe9fd4q', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:43751/status,
Dashboard: http://127.0.0.1:43751/status,Workers: 10
Total threads: 20,Total memory: 361.35 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37337,Workers: 10
Dashboard: http://127.0.0.1:43751/status,Total threads: 20
Started: Just now,Total memory: 361.35 GiB
Comm: tcp://127.0.0.1:45827,Total threads: 2
Dashboard: http://127.0.0.1:45840/status,Memory: 36.13 GiB
Nanny: tcp://127.0.0.1:44294,


In [3]:
## read nino3.4 index
READ_SAVE_NINO3_4=False
if READ_SAVE_NINO3_4:
    from indian_monsoon_analys import convert_nino34_indx_to_netcdf
    nino34_file='/neelin2020/Indian_Monsoon_files/nino34.long.anom.data'
    SAVE_FILE='/neelin2020/Indian_Monsoon_files/nino34_indx_monthly.nc'
    convert_nino34_indx_to_netcdf(nino34_file,SAVE_FILE)

In [4]:
## set paths to files 
main_file_path='/neelin2020/Indian_Monsoon_files/'
fil_rain=glob.glob(main_file_path+'imd_rain_daily_0p25_monsoon-masked_1901-2020.nc')[0]
fil_nino34=glob.glob(main_file_path+'nino34_noaa-ersst_monthly.nc')[0]

In [5]:
## generate list of era5 files
era5_file_path=main_file_path+'/ERA5/'
fils_era5=glob.glob(era5_file_path+'era-5_Tq_*grib')
fils_era5_idx=glob.glob(era5_file_path+'era-5_Tq_*idx')
fils_era5.sort()
fils_era5=[fil for fil in fils_era5 if fil not in fils_era5_idx]

In [6]:
## read land mask
land_mask=xr.open_dataset(fil_rain).land_mask

In [7]:
fil_cmz_mask_regridded='/neelin2020/Indian_Monsoon_files/cmz_mask_regridded.nc'
### regrid cmz mask if required
REGRID_SAVE_CMZ=False

if REGRID_SAVE_CMZ:
    fil_cmz_mask=glob.glob(main_file_path+'imd_0.25_CMZ_mask.nc')[0]
    from indian_monsoon_analys import regrid_cmz_mask
    cmz_mask=xr.open_dataset(fil_cmz_mask)
    ## read and regrid cmz mask if necessary
    cmz_mask_regridded=regrid_cmz_mask(cmz_mask,land_mask,fil_cmz_mask)
    cmz_mask.close()


In [8]:
## set era5 options
dir_out='/neelin2020/Indian_Monsoon_files/buoy_files/'
filter_surf_press=750
save_path='/neelin2020/Indian_Monsoon_files/processed_thermo/'

In [9]:
import indian_monsoon_analys
import importlib
importlib.reload(indian_monsoon_analys)
from indian_monsoon_analys import ProcessIMD, MaskIndianMonsoon,ProcessERA5

In [ ]:
for fil in fils_era5:
    date_str=fil.split('Tq_')[-1].split('.grib')[0]
    month=dt.datetime.strptime(date_str,'%Y_%m_%d').month
    if month in range(6,9): ## JJAS filter
        print(fil)
        obj_era5=ProcessERA5(fil,dir_out,filter_surf_press,land_mask,fil_cmz_mask_regridded,save_path)
        obj_era5.main()
        break
